In [1]:
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import concatenate
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import glob
import cv2
import locale
import os

### Load in Shankar array Data

In [46]:
shankarrs = []

with open('data-sa.csv') as f:
    for line in f.readlines():
        shankarrs.append(eval(line))

shankarrs = np.array(shankarrs)
shankarrs.tofile('shankarrs-np.dat')
shankarrs

array([[[ 0,  0,  0, ...,  0, -2,  0],
        [ 0,  0, -2, ..., -2,  0, -2],
        [ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0]],

       [[ 0, -2,  0, ..., -1,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0, -2],
        [ 0,  0,  0, ..., -2,  0,  0]],

       [[ 0,  0,  0, ...,  1, -2,  0],
        [ 0,  0, -2, ..., -2,  0, -2],
        [ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0]],

       ...,

       [[ 0,  0, -2, ..., -1, -1, -1],
        [ 0, -2, -1, ..., -1, -1, -1],
        [ 0,  0, -2, ..., -1, -1, -1],
        [ 0,  0,  0, ..., -1, -1, -1]],

       [[ 0,  0,  0, ..., -1, -1, -1],
        [-2, -1, -2, ..., -1, -1, -1],
        [-2, -2,  1, ..., -1, -1, -1],
        [-1, -2, -2, ..., -1, -1, -1]],

       [[ 0,  0, -2, ..., -1, -1, -1],
        [ 0, -2, -1, ..., -1, -1, -1],
        [ 0,  0, -2, ..., -1, -1, -1],
        [ 0,  0,  0, ..., -1, -1, -1]]])

### Load in player hand data

In [48]:
player_hands = []

with open('data-ph.csv') as f:
    for line in f.readlines():
        player_hands.append(eval(line))

player_hands = np.array([arr.flatten() for arr in np.array(player_hands)])
player_hands

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0]])

In [49]:
player_hands.tofile('player-hands-np.dat')

### split train and test data

In [50]:
trainX, testX, trainY, testY = train_test_split(shankarrs, player_hands, test_size=0.25, random_state=42)

### Create CNN model

In [51]:
height = 4
width = 13
depth = 1
filters = (52,52)

inputShape = (height, width, depth)
chanDim = -1

inputs = Input(shape=inputShape)

for (i, f) in enumerate(filters):
    # if this is the first CONV layer then set the input appropriately
    if i == 0:
        x = inputs

    # CONV => RELU => BN => POOL
    x = Conv2D(f, (3, 3), padding="same")(x)
    x = Activation("relu")(x)
    x = BatchNormalization(axis=chanDim)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

x = Flatten()(x)
x = Dense(52)(x)
x = Activation("relu")(x)
x = BatchNormalization(axis=chanDim)(x)
x = Dropout(0.5)(x)

cnn = Model(inputs, x)

### Compile

In [52]:
opt = Adam(lr=1e-3, decay=1e-3 / 200)
cnn.compile(loss="mean_absolute_percentage_error", optimizer=opt)

### Train

In [ ]:
cnn.fit(
    trainX, trainY,
    validation_data=(testX, testY),
    epochs=200, batch_size=25)

Epoch 1/200
76925/76925 [==============================] - 434s 6ms/step - loss: 1573957.1250 - val_loss: 93710.6719
Epoch 2/200
57285/76925 [=====================>........] - ETA: 1:43 - loss: 77598.3906

### Testing

In [43]:
print(cnn.predict(np.array([testX[0]])))

[[ 2.6450689e-05 -1.6463759e-06  4.5798766e-05 -4.3367007e-05
  -5.7580437e-06 -1.6109745e-05 -2.9030840e-05  3.5229328e-05
   1.2055444e-05 -4.7313679e-06 -1.3495663e-05 -6.4264939e-05
   7.0859242e-05  7.4503405e-06 -9.7842712e-06 -1.5950505e-05
   8.2238075e-06  5.2024548e-05  1.7929604e-05  9.4108882e-06
   4.2935826e-06 -2.0340838e-05  9.8186902e-05 -4.1662410e-05
   1.3393825e-05 -7.4788363e-06  4.5096887e-05 -2.1559670e-05
   4.1357773e-05  2.8861545e-05  5.2784839e-05  4.3548749e-05
  -1.8538874e-05 -2.3668874e-05 -9.6970098e-07 -4.2419146e-05
   1.5787009e-08 -2.4660367e-05  1.8387942e-05 -6.0266107e-06
   2.0064545e-05 -1.6537069e-07  3.3888711e-05 -2.3833363e-05
  -1.4311710e-05  5.9948688e-05  2.1980055e-05  7.8635148e-06
  -2.5515983e-05 -1.7158440e-05 -2.7229140e-05 -2.2183141e-05]]


In [45]:
print(testY[0])

[0 1 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 1 0 0 1 0 1 0 0 0 0 1 1 0]


In [49]:
model.save('sankarr.h5')